<a href="https://colab.research.google.com/github/Luckyv21/QuickCart/blob/main/Module%202%20-%20Deep%20Learning/2.1%20Car%20Price%20Prediction%20using%20FNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car Price Prediction using Feedforward Neural Network
This notebook demonstrates the process of building a Feedforward Neural Network (FNN) to predict car prices using the provided dataset. We will go through data loading, cleaning, preprocessing, model building, training, evaluation, and prediction.

## Data Loading
We begin by loading the dataset using pandas.

## Data Cleaning
We check for missing values and drop unnecessary columns if any.

In [45]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("CarPrice_dataset.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [46]:
df.shape

(205, 26)

In [47]:
df.info()
#df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [48]:
df.isnull().sum()

,0
car_ID,0
symboling,0
CarName,0
fueltype,0
aspiration,0
doornumber,0
carbody,0
drivewheel,0
enginelocation,0
wheelbase,0


In [49]:
for i in df.select_dtypes(include=['object']).columns:
  print(i, len(df[i].unique()), df[i].unique().tolist())

CarName 147 ['alfa-romero giulia', 'alfa-romero stelvio', 'alfa-romero Quadrifoglio', 'audi 100 ls', 'audi 100ls', 'audi fox', 'audi 5000', 'audi 4000', 'audi 5000s (diesel)', 'bmw 320i', 'bmw x1', 'bmw x3', 'bmw z4', 'bmw x4', 'bmw x5', 'chevrolet impala', 'chevrolet monte carlo', 'chevrolet vega 2300', 'dodge rampage', 'dodge challenger se', 'dodge d200', 'dodge monaco (sw)', 'dodge colt hardtop', 'dodge colt (sw)', 'dodge coronet custom', 'dodge dart custom', 'dodge coronet custom (sw)', 'honda civic', 'honda civic cvcc', 'honda accord cvcc', 'honda accord lx', 'honda civic 1500 gl', 'honda accord', 'honda civic 1300', 'honda prelude', 'honda civic (auto)', 'isuzu MU-X', 'isuzu D-Max ', 'isuzu D-Max V-Cross', 'jaguar xj', 'jaguar xf', 'jaguar xk', 'maxda rx3', 'maxda glc deluxe', 'mazda rx2 coupe', 'mazda rx-4', 'mazda glc deluxe', 'mazda 626', 'mazda glc', 'mazda rx-7 gs', 'mazda glc 4', 'mazda glc custom l', 'mazda glc custom', 'buick electra 225 custom', 'buick century luxus (sw)

In [50]:
class ModifiedLabelEncoder(LabelEncoder):
  def fit_transform(self, y, *args, **kwargs):
    for i in y:
      y[i] = super().fit_transform(y[i]).reshape(-1, 1)
      return y

  def transform(self, y, *args, **kwargs):
    for i in y:
      y[i] = super().transform(y[i]).reshape(-1, 1)
      return y

## Data Preprocessing
We handle categorical variables using one-hot encoding and normalize numerical features.

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

# Separate features and target
X = df.drop(columns=['price'])
y = df['price']

# Identify categorical and numerical columns
le = [i  for i in df.select_dtypes(include=['object']).columns if len(df[i].unique()) > 2]

ohe = [i for i in df.select_dtypes(include=['object']).columns if len(df[i].unique()) ==2]

num = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

num.pop()
transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num),
        ('ohe', OneHotEncoder(drop='first'), ohe),
        ('le', ModifiedLabelEncoder(), le)
    ],
    remainder='passthrough'
)
data = transformer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((164, 25), (41, 25), (164,), (41,))

In [72]:
X_train.shape

(164, 25)

## Model Building
We build a Feedforward Neural Network using TensorFlow/Keras.

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


# Define the model
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='relu'))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics = ["mean_absolute_error", "mean_squared_error"])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_49 (Dense)                │ (None, 256)            │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,561 (197.50 KB)

 Trainable params: 50,561 (197.50 KB)

 Non-trainable params: 0 (0.00 B)

## Model Training
We train the model using the training data.

In [74]:
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=0)

ValueError: Invalid dtype: object

## Model Evaluation
We evaluate the model using the test data.

In [70]:
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score

y_pred = model.predict(X_test)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

ValueError: Invalid dtype: object

## Prediction on New Data
We demonstrate prediction using new sample data.

In [62]:
# Example new data (replace with actual values from dataset)
new_data = X.iloc[[0]]
new_data_processed = preprocessor.transform(new_data)

predicted_price = model.predict(new_data_processed)
print(f"Predicted Price: {predicted_price[0][0]}")

NameError: name 'preprocessor' is not defined